Let's import the libraries needed

In [10]:
import numpy as np 
!conda install -c conda-forge pandas --yes 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
!conda install -c conda-forge lxml --yes 
import json
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium 
!conda install -c conda-forge geocoder --yes
import geocoder
import time
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    click-7.1.2                |     pyh9f0ad1d_0          64 KB  conda-forge
    future-0.18.2              |   py36h9f0ad1d_1         714 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                          

Let's obtain the dataframe as done in the previous notebook/question

In [11]:
to_scrape_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(to_scrape_url)[0]
df['Borough'] = df['Borough'].replace('Not assigned', np.nan)
df = df.dropna().reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [13]:
lats = np.zeros(len(df))
longs = np.zeros(len(df))
# get latitude and longitude for each postal code
for i in range(len(df)):
    lat_lng_coords = None
    postal_code = df.iloc[i, 0]
    print(i, postal_code)
    k = 0
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        time.sleep(0.01) # add a little delay not to waste calls
        k += 1
        if(k > 50):
            print('Data collection failed')
            break
    lats[i] = lat_lng_coords[0]
    longs[i] = lat_lng_coords[1]
# Add lat and long to dataframe
df['Latitude'] = lats
df['Longitude'] = longs
df.head()

0 M3A
Data collection failed


TypeError: 'NoneType' object is not subscriptable

Since geocode library does not appear to work I will load in the location data from a csv file. and merge it to my dataframe after making sure they are in the same order

In [16]:
loc_data_url = 'http://cocl.us/Geospatial_data'
loc_data = pd.read_csv(loc_data_url)
df = df.sort_values('Postal Code').reset_index(drop=True)
loc_data = loc_data.sort_values('Postal Code').reset_index(drop=True)
are_equal = (df['Postal Code'] == loc_data['Postal Code']).all()
print('Do the two datframes correspond? {}'.format(are_equal))

Do the two datframes correspond? True


If they correspond let's merge them, otherwise we output and error

In [19]:
if are_equal:
    boroughs = df.merge(loc_data, 'left') 
else:
    print('Cannot merge the dataframe with location data because the postal codes do no correspond.')
boroughs.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
